## generate human path from human model

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import random
import pandas as pd
import itertools
import math
# from mdp import GridworldMDP

from collections import deque
# from utils_suboptimal import *
import random
import argparse

from utils_human_exp import *

In [2]:
## load human model
human_model = torch.load('models/human_model_grid10.pt', map_location=torch.device('cpu'))
human_model.eval()

HumanNN(
  (fc1): Linear(in_features=400, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=4, bias=True)
  (sm): Softmax(dim=1)
)

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

grid_size = 6
num_goal_positions = 2
max_envs = 10


In [4]:
def continuous_design_wcd(grid_size, num_goal_positions, max_iterations=100):
    """
    Continuously creates designs and computes WCD.
    Ignores environments where the shortest path to any goal is -1.
    """

    def is_valid_environment(paths):
        """ Check if the environment is valid (no shortest path is -1) """
        return all(len(path) > 0 for path in paths)

    wcd_results = []
    designs = []
    count = 0
    valid_counts = 0
    for _ in range(max_iterations):
        num_blocked_positions = random.choice(range(grid_size*2))
        new_goal_positions, new_special_reward_positions, new_blocked_positions, start_pos = randomize_pos(grid_size, 
                                                                                                           num_goal_positions, 0, num_blocked_positions)
        is_valid = is_design_valid(grid_size, new_goal_positions, new_blocked_positions, start_pos)
        if not is_valid:
            continue
        valid_counts+=1
        wcd = compute_human_wcd(model = human_model, grid_size=grid_size, goal_positions = new_goal_positions, 
                                blocked_positions = new_blocked_positions, start_pos = start_pos,search_depth = 19)
        
        #compute_human_wcd(model, grid_size, goal_positions, blocked_positions, start_pos, search_depth =19)
        x= encode_grid_design(grid_size, new_goal_positions, new_blocked_positions, start_pos).squeeze()
        
        grid = decode_grid_design(x,return_map = True)
        
        print(wcd)
        # plot_grid(grid)
        if wcd is None:
            count+=1
            wcd_results.append(wcd)
            designs.append(encode_grid_design(grid_size, new_goal_positions, new_blocked_positions, start_pos))
    print("Percent failed",count/valid_counts)
    return designs,wcd_results



In [5]:
for i in range(0, 5):
    grid_size = 10
    num_goal_positions = 2
    max_iterations = 20  # Set the number of iterations
    x,y = continuous_design_wcd(grid_size, num_goal_positions, max_iterations)
    print(y)
    file_name = f"data/grid{grid_size}/model_training/tmp_human_path.pt"
    # update_or_create_dataset(file_name, x, y)
    # except:
    #     pass

14
4
16
10
16
15
0
8
0
5
6
11
6
4
0
12
7
0
6
10
Percent failed 0.0
[]
0
5
5
8
0
12
0
0
4
7
10
14
0
12
3
13
0
10
11
6
Percent failed 0.0
[]
0
0
0
0
9
0
7
16
0
0
14
4
0
8
0
12
3
0
3
8
Percent failed 0.0
[]
10
10
5
0
6
9
14
5
0
0
2
16
0
17
5
4
0
11
0
0
Percent failed 0.0
[]
0
0
0
0
8
6
0
8
6
0
12
10
0
16
8
7
0
7
12
10
Percent failed 0.0
[]
